# Modeling and Training MNIST Dataset Using TensorFlow GPU

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/the-deep-learners/deep-learning-illustrated/blob/master/notebooks/vggnet_in_keras.ipynb)

#### Set seed for reproducibility

#### Load dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import time

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Load *and preprocess* data

In [3]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data(path="mnist.npz")
print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
X_train = X_train.reshape (60000, 28, 28, 1).astype('float32')
X_test = X_test.reshape (10000, 28, 28, 1).astype('float32')
X_train /= 255
X_test /= 255

In [5]:
n_classes = 10
Y_train = tf.keras.utils.to_categorical(Y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, n_classes)

#### Design neural network architecture

In [6]:
time.sleep(10)

In [7]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(2, 2), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())


model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())


model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(BatchNormalization())


model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(n_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 64)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 25, 25, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 21, 21, 128)       512       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 256)       2

#### Configure model

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Configure TensorBoard 

In [10]:
tensorbrd = TensorBoard('logs/vggnet')

#### Train!

In [11]:
model.fit(X_train, Y_train, batch_size=128, epochs=5, verbose=1, validation_data=(X_test, Y_test), shuffle=True, callbacks=[tensorbrd])

Epoch 1/5
  1/469 [..............................] - ETA: 0s - loss: 2.8319 - accuracy: 0.1094WARNING:tensorflow:From C:\Users\hle\.conda\envs\cnnvenv\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
469/469 [==============================] - 35s 75ms/step - loss: 0.1861 - accuracy: 0.9564 - val_loss: 0.1825 - val_accuracy: 0.9448
Epoch 2/5
469/469 [==============================] - 34s 73ms/step - loss: 0.0753 - accuracy: 0.9799 - val_loss: 0.1174 - val_accuracy: 0.9764
Epoch 3/5
469/469 [==============================] - 34s 74ms/step - loss: 0.0658 - accuracy: 0.9837 - val_loss: 0.0531 - val_accuracy: 0.9875
Epoch 4/5
469/469 [==============================] - 35s 74ms/step - loss: 0.0540 - accuracy: 0.9862 - val_loss: 0.0517 - val_accuracy: 0.9883
Epoch 5/5
469/469 [==============================] - 3

In [12]:
model.save("final_model.h5")